# Modelo Preditivo para descobrir qual vai ser o próximo hit do Sportify - Marcelo Rossignolli

In [50]:
#Importa as bibliotecas permitidas 
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  

%matplotlib inline

## 1. Limpeza e Tratamento de Valores Nulos

In [51]:
#Define o dataset de treino e teste
df_test = pd.read_csv("test.csv")

df_train = pd.read_csv("train.csv")

### 1.1 Verificação de valores nulos:

&nbsp;&nbsp;&nbsp;&nbsp;Antes de iniciar o pré-processamento, verifiquei a existência de valores nulos nas colunas do dataset.

In [52]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())

track_unique_id      0
track_id             0
artists              0
album_name           0
track_name           0
duration_ms          0
explicit             0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
track_genre          0
popularity_target    0
dtype: int64
track_unique_id     0
track_id            0
artists             1
album_name          1
track_name          1
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


### 1.2 Tratamento de valores nulos:

&nbsp;&nbsp;&nbsp;&nbsp;Para colunas numéricas com valores nulos: substituí valores ausentes pela mediana de cada coluna, uma vez que a mediana é menos sensível a outliers.

&nbsp;&nbsp;&nbsp;&nbsp;Para colunas categóricas: Os valores nulos foram substituídos por 'unknown', garantindo que esses dados possam ser codificados sem perda de informação.

In [53]:
# Substituindo valores nulos nas colunas numéricas
numeric_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
df_train[numeric_columns] = df_train[numeric_columns].fillna(df_train[numeric_columns].median())

# Substituindo valores nulos nas colunas categóricas
categorical_columns = df_train.select_dtypes(include=['object', 'bool']).columns
df_train[categorical_columns] = df_train[categorical_columns].fillna('unknown')


### 1.3 Tratamento de outliers:

&nbsp;&nbsp;&nbsp;&nbsp;Os outliers foram detectados e tratados usando o método de IQR, limitando os valores ao intervalo entre o percentil 1 e 99.

In [54]:
# Tratamento de outliers
Q1 = df_train[numeric_columns].quantile(0.25)
Q3 = df_train[numeric_columns].quantile(0.75)
IQR = Q3 - Q1

df_train = df_train[~((df_train[numeric_columns] < (Q1 - 1.5 * IQR)) | (df_train[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]


## 2. Codificação de Variáveis Categóricas

&nbsp;&nbsp;&nbsp;&nbsp;Utilizei OneHotEncoder para transformar as variáveis categóricas em formato numérico adequado para o modelo.

In [55]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Aplicando o OneHotEncoder nas colunas categóricas
categorical_columns = ['track_id', 'artists', 'album_name', 'track_name', 'explicit', 'track_genre']
numeric_columns = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode',
                   'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

# Definindo o pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])


## 3. Exploração e Visualização dos Dados

&nbsp;&nbsp;&nbsp;&nbsp;A exploração dos dados fpoi feita utilizando métodos Pandas como describe() e value_counts().

### 3.1 Distribuição da variável alvo (popularity_target):

In [56]:
# Distribuição da variável alvo
print(df_train['popularity_target'].value_counts())

popularity_target
1    23321
0    20771
Name: count, dtype: int64


### 3.2 Estatísticas descritivas das variáveis numéricas:

In [57]:
# Estatísticas descritivas
print(df_train.describe())

       track_unique_id    duration_ms  danceability        energy  \
count     44092.000000   44092.000000  44092.000000  44092.000000   
mean      58521.654813  216280.139663      0.593321      0.671343   
std       32154.042899   55803.002597      0.147202      0.212995   
min           0.000000   44040.000000      0.098900      0.009940   
25%       31439.750000  178373.000000      0.495000      0.522000   
50%       61214.500000  211920.000000      0.599000      0.704000   
75%       86391.250000  249643.250000      0.701000      0.849000   
max      113998.000000  391895.000000      0.984000      1.000000   

                key      loudness          mode   speechiness  acousticness  \
count  44092.000000  44092.000000  44092.000000  44092.000000  44092.000000   
mean       5.301143     -6.912627      0.658963      0.055647      0.276512   
std        3.560273      3.087871      0.474063      0.029563      0.294876   
min        0.000000    -17.514000      0.000000      0.022100 

## 4. Formulação de Hipóteses 

- Hipótese 1: Músicas com maior "danceability" tendem a ser mais populares.

- Hipótese 2: Músicas explicitamente marcadas como "explicit=True" têm mais chances de ser populares.

- Hipótese 3: Músicas com valores mais altos de "energy" são mais propensas a se tornarem populares.

## 5. Seleção de Features (até 1,0 pt)

&nbsp;&nbsp;&nbsp;&nbsp;Com base nas hipóteses e na análise exploratória, as seguintes features foram escolhidas para o modelo:

- Numéricas: "danceability", "energy", "tempo", "loudness", "acousticness".

- Categóricas: "explicit", "track_genre", "album_name".

## 6. Construção e Avaliação do Modelo 

&nbsp;&nbsp;&nbsp;&nbsp;Utilizamos o RandomForestClassifier para a previsão da popularidade.

In [58]:
# Verificando se a coluna 'track_id' realmente existe e ajustando conforme necessário
categorical_columns = ['artists', 'album_name', 'track_name', 'explicit', 'track_genre']
numeric_columns = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode',
                   'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

# Se 'track_id' não existe, remova-a
X = df_train.drop(columns=['popularity_target', 'track_unique_id'])  # Remova apenas as colunas que realmente existem
y = df_train['popularity_target']

# Dividindo os dados em treino e validação
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicando o pré-processamento e treinando o modelo
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

# Preprocessamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])

# Aplicando o pré-processamento
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

# Treinando o modelo
model = RandomForestClassifier(random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Avaliando o modelo
print("Acurácia no conjunto de validação:", model.score(X_val, y_val))


Acurácia no conjunto de validação: 0.8567864837283139


## 7. Geração do Arquivo de Submissão

&nbsp;&nbsp;&nbsp;&nbsp;Após realizar as previsões no conjunto de teste, geramos o arquivo CSV contendo o track_unique_id e a predição do popularity_target.

In [62]:
# Preservar a coluna 'track_unique_id' para o arquivo de submissão
track_unique_id = df_test['track_unique_id'].copy()

# Remover as colunas 'track_id' e 'track_unique_id' antes de passar os dados ao modelo
df_test_features = df_test.drop(columns=['track_id', 'track_unique_id'], errors='ignore')

# Fazendo previsões com o pipeline já treinado
test_data_predictions = model_pipeline.predict(df_test_features)

# Criando o DataFrame de submissão, usando a coluna 'track_unique_id' preservada
submission = pd.DataFrame({
    'track_unique_id': track_unique_id,
    'popularity_target': test_data_predictions
})

# Salvando o resultado no arquivo CSV
submission.to_csv('modelo.csv', index=False)

print("Arquivo salvo com sucesso.")


Arquivo salvo com sucesso.
